In [11]:
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import playercareerstats, playergamelog, teamgamelog, leaguegamefinder, leaguegamelog, commonteamroster, teamdashlineups, commonplayerinfo
from nba_api.live.nba.endpoints import scoreboard, playbyplay, boxscore
from nba_api.stats.library.parameters import SeasonAll
from datetime import datetime, timezone
from dateutil import parser
import pandas as pd
from pprint import pprint
from IPython.display import display
import requests
import time

In [3]:
target_player_name = "LeBron James"

# Find Players ID ---------------------------------------------------------------------------------------------------
nba_players = players.get_players() # Get the list of all NBA players
player_id = next((x for x in players.get_players() if x.get("full_name") == target_player_name), None).get("id")
#--------------------------------------------------------------------------------------------------------------------


# Get players team abbreviation -------------------------------------------------------------------------------------
player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id) # Get common player info for the specified player_id
player_info_data = player_info.get_data_frames()[0]
# Extract the team abbreviation from the player details
team_abbreviation = player_info_data['TEAM_ABBREVIATION'].values[0]
#--------------------------------------------------------------------------------------------------------------------


# Get the player's game log -----------------------------------------------------------------------------------------
gamelog = pd.concat(playergamelog.PlayerGameLog(player_id=player_id, season=SeasonAll.all).get_data_frames())
gamelog["GAME_DATE"] = pd.to_datetime(gamelog["GAME_DATE"], format="%b %d, %Y")
#--------------------------------------------------------------------------------------------------------------------


# Initialize game finder --------------------------------------------------------------------------------------------
gamefinder = leaguegamefinder.LeagueGameFinder()
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
#--------------------------------------------------------------------------------------------------------------------


# Add additional columns to df --------------------------------------------------------------------------------------
gamelog['PLAYER_NAME'] = target_player_name  # Add a column for the player's name 
#--------------------------------------------------------------------------------------------------------------------



# Display the updated DataFrame
gamelog = gamelog[['PLAYER_NAME', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'MIN','PTS', 'FGM', 'FGA', 'FG_PCT', 'FTM', 'FTA', 'FT_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'REB', 'PLUS_MINUS']]
print(team_abbreviation)
gamelog

LAL


,PLAYER_NAME,Game_ID,GAME_DATE,MATCHUP,MIN,PTS,FGM,FGA,FG_PCT,FTM,FTA,FT_PCT,FG3M,FG3A,FG3_PCT,REB,PLUS_MINUS
0,LeBron James,0022300242,2023-11-25,LAL @ CLE,36,22,8,23,0.348,5,7,0.714,1,9,0.111,6,2
1,LeBron James,0022300238,2023-11-22,LAL vs. DAL,36,26,9,19,0.474,4,7,0.571,4,9,0.444,9,7
2,LeBron James,0022300042,2023-11-21,LAL vs. UTA,24,17,6,10,0.600,2,2,1.000,3,5,0.600,7,14
3,LeBron James,0022300216,2023-11-19,LAL vs. HOU,40,37,14,19,0.737,7,9,0.778,2,5,0.400,6,2
4,LeBron James,0022300036,2023-11-17,LAL @ POR,35,35,13,22,0.591,4,6,0.667,5,9,0.556,5,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432,LeBron James,0020300068,2003-11-07,CLE @ IND,44,23,8,18,0.444,6,7,0.857,1,2,0.500,5,-7
1433,LeBron James,0020300057,2003-11-05,CLE vs. DEN,41,7,3,11,0.273,1,1,1.000,0,2,0.000,11,-3
1434,LeBron James,0020300038,2003-11-01,CLE @ POR,39,8,3,12,0.250,2,2,1.000,0,1,0.000,4,-21
1435,LeBron James,0020300018,2003-10-30,CLE @ PHX,41,21,8,17,0.471,4,7,0.571,1,5,0.200,12,-3


In [16]:
import pandas as pd
from nba_api.stats.endpoints import commonplayerinfo, playergamelog
from requests.exceptions import ReadTimeout

def get_player_avg_pts(player_name, season='2023-24'):
    try:
        # Find Players ID
        nba_players = players.get_players()
        player_id = next((x['id'] for x in nba_players if x.get("full_name") == player_name), None)

        # Get player's team abbreviation
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
        team_abbreviation = player_info.get_data_frames()[0]['TEAM_ABBREVIATION'].values[0]

        # Get the player's game log with a timeout of 10 seconds
        gamelog = pd.concat(playergamelog.PlayerGameLog(player_id=player_id, season=season, timeout=10).get_data_frames())
        gamelog["GAME_DATE"] = pd.to_datetime(gamelog["GAME_DATE"], format="%b %d, %Y")

        # Add additional columns to df
        gamelog['PLAYER_NAME'] = player_name

        # Create a new DataFrame with selected columns
        selected_columns = ['PLAYER_NAME', 'PTS']
        new_df = gamelog[selected_columns]

        # Calculate average PTS
        average_pts = new_df['PTS'].mean()

        # Return a dictionary with player information
        return {'PLAYER_NAME': player_name, 'TeamAbbreviation': team_abbreviation, 'Average PTS': average_pts}

    except JSONDecodeError as e:
        print(f"JSONDecodeError processing data for {player_name}: {e}")
        return None

    except ReadTimeout as e:
        print(f"Timeout error processing data for {player_name}: {e}")
        return None

    except Exception as e:
        print(f"Error processing data for {player_name}: {e}")
        return None


In [12]:
# GET ALL NBA TEAMS INFO

# Get the list of all NBA teams
nba_teams = teams.get_teams()
teams_df = pd.DataFrame(nba_teams)
# Rename the 'id' column to 'teamID'
teams_df.rename(columns={'id': 'TeamID'}, inplace=True)

# GET ALL PLAYERS INFO

# Retrieve the roster data for all teams
teams_roster = []

for team_id in teams_df['TeamID']:
    player_stats_data = commonteamroster.CommonTeamRoster(season='2023-24', team_id=team_id)
    df = player_stats_data.common_team_roster.get_data_frame()
    
    # Add the TeamAbbreviation column to the original DataFrame 'df'
    df['TeamAbbreviation'] = teams_df.loc[teams_df['TeamID'] == team_id, 'abbreviation'].iloc[0]
    
    teams_roster.append(df)

# Concatenate all the DataFrames in teams_roster list
df_combined = pd.concat(teams_roster, ignore_index=True)
df_combined.tail()

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED,TeamAbbreviation
527,1610612766,2023,00,Gordon Hayward,Gordon,gordon-hayward,20,F,6-7,225,"MAR 23, 1990",33.0,13,Butler,202330,Traded from BOS on 11/29/20,CHA
528,1610612766,2023,00,JT Thor,JT,jt-thor,21,F,6-9,203,"AUG 26, 2002",21.0,2,Auburn,1630550,Draft Rights Traded from DET on 08/06/21,CHA
529,1610612766,2023,00,Brandon Miller,Brandon,brandon-miller,24,F,6-7,200,"NOV 22, 2002",21.0,R,Alabama,1641706,#2 Pick in 2023 Draft,CHA
530,1610612766,2023,00,P.J. Washington,P.J.,pj-washington,25,F,6-7,230,"AUG 23, 1998",25.0,4,Kentucky,1629023,#12 Pick in 2019 Draft,CHA
531,1610612766,2023,00,Frank Ntilikina,Frank,frank-ntilikina,44,G,6-4,200,"JUL 28, 1998",25.0,6,Strasbourg IG,1628373,Signed on 08/05/23,CHA


In [ ]:
# Apply the function to every player in the DataFrame
result_list = df_combined['PLAYER'].apply(lambda player: get_player_avg_pts(player))

# Create a DataFrame from the list of dictionaries
result_df = pd.DataFrame(result_list)
result_df

In [ ]:
import pandas as pd
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players

# Get the list of all NBA players
nba_players = players.get_players()

# Initialize an empty list to store individual player career DataFrames
career_stats_list = []

# Loop through all players and retrieve their career stats
for player in nba_players:
    player_id = player['id']
    player_name = player['full_name']
    
    # Retrieve career stats for the current player
    career_stats = playercareerstats.PlayerCareerStats(player_id=player_id)
    career_per_season_data = career_stats.get_data_frames()[0]
    
    # Create a DataFrame for the player's career stats
    player_career_stats_df = pd.DataFrame(career_per_season_data)
    
    # Add player information to the DataFrame
    player_career_stats_df['PLAYER_NAME'] = player_name
    
    # Append the DataFrame to the list
    career_stats_list.append(player_career_stats_df)

# Concatenate all individual player career DataFrames into a single DataFrame
all_players_career_df = pd.concat(career_stats_list, ignore_index=True)

# Display the resulting DataFrame
print(all_players_career_df)
